In [1]:
import os

# Create the folder if it doesn't exist
os.makedirs("midi_songs", exist_ok=True)

In [2]:
from google.colab import files

# Upload MIDI files from your computer
uploaded = files.upload()


Saving PartyRockAnthem(2).mid to PartyRockAnthem(2).mid
Saving Dreams.mid to Dreams.mid
Saving BeingThere.mid to BeingThere.mid
Saving DearestHelpless.mid to DearestHelpless.mid
Saving Wing207.mid to Wing207.mid
Saving DonJuanDemarco.mid to DonJuanDemarco.mid
Saving AsAnyFoolCanSee.mid to AsAnyFoolCanSee.mid
Saving Rememberthename(1).mid to Rememberthename(1).mid
Saving SoulsNeeds.mid to SoulsNeeds.mid
Saving LookWhatHappened.mid to LookWhatHappened.mid
Saving TheLegatoTechnique.mid to TheLegatoTechnique.mid
Saving WithoutYou.mid to WithoutYou.mid
Saving 26Cents.mid to 26Cents.mid
Saving SomeoneLikeYou.mid to SomeoneLikeYou.mid


In [3]:
import shutil

# Move all uploaded files to the midi_songs folder
for filename in uploaded.keys():
    shutil.move(filename, f"midi_songs/{filename}")

In [4]:
os.listdir("midi_songs")

['BeingThere.mid',
 '26Cents.mid',
 'WithoutYou.mid',
 'Dreams.mid',
 'Wing207.mid',
 'Rememberthename(1).mid',
 'DonJuanDemarco.mid',
 'SoulsNeeds.mid',
 'SomeoneLikeYou.mid',
 'LookWhatHappened.mid',
 'DearestHelpless.mid',
 'TheLegatoTechnique.mid',
 'PartyRockAnthem(2).mid',
 'AsAnyFoolCanSee.mid']

In [5]:
from music21 import converter, instrument, note, chord

def get_notes_from_midi(folder_path):
    notes = []
    for file in os.listdir(folder_path):
        if file.endswith(".mid"):
            midi = converter.parse(os.path.join(folder_path, file))

            parts = instrument.partitionByInstrument(midi)
            if parts:
                notes_to_parse = parts.parts[0].recurse()
            else:
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

notes = get_notes_from_midi("midi_songs")
print(f"Total notes/chords extracted: {len(notes)}")


Total notes/chords extracted: 9000


In [6]:
import numpy as np
from keras.utils import to_categorical

sequence_length = 100

# Map notes to integers
note_names = sorted(set(notes))
note_to_int = {note: num for num, note in enumerate(note_names)}

network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    seq_out = notes[i + sequence_length]
    network_input.append([note_to_int[n] for n in seq_in])
    network_output.append(note_to_int[seq_out])

n_patterns = len(network_input)
n_vocab = len(note_names)

# Reshape and normalize input
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)

# One-hot encode output
network_output = to_categorical(network_output)


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation

model = Sequential()
model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the model
model.fit(network_input, network_output, epochs=10, batch_size=64)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 11s 49ms/step - loss: 4.5394
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - loss: 4.0130
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - loss: 3.8422
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - loss: 3.7642
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - loss: 3.6992
Epoch 6/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - loss: 3.8059
Epoch 7/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - loss: 3.5108
Epoch 8/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 10s 51ms/step - loss: 3.2697
Epoch 9/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 10s 51ms/step - loss: 3.0036
Epoch 10/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - loss: 2.9456


In [8]:
import random

int_to_note = {num: note for note, num in note_to_int.items()}

start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]
pattern = pattern.reshape(1, sequence_length, 1)

prediction_output = []

for note_index in range(500):
    prediction = model.predict(pattern, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    pattern = np.append(pattern[:, 1:, :], [[[index / float(n_vocab)]]], axis=1)


In [9]:
from music21 import stream, note, chord

offset = 0
output_notes = []

for pattern in prediction_output:
    if '.' in pattern:
        chord_notes = [note.Note(int(n)) for n in pattern.split('.')]
        new_chord = chord.Chord(chord_notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        output_notes.append(new_note)
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='generated_music.mid')


'generated_music.mid'

In [10]:
from google.colab import files
files.download('generated_music.mid')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>